In [3]:
import os
import time
import logging
from dataclasses import dataclass, field
from typing import List, Optional, Dict, Tuple

import laspy
import numpy as np
from scipy.spatial import cKDTree
from sklearn.neighbors import KDTree
from sklearn.cluster import DBSCAN

try:
    from hdbscan import HDBSCAN
    HAS_HDBSCAN = True
except ImportError:
    HAS_HDBSCAN = False

import open3d as o3d

try:
    from shapely.geometry import shape
    from shapely.vectorized import contains
    import fiona
    HAS_SHAPELY = True
except ImportError:
    HAS_SHAPELY = False

logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s")
log = logging.getLogger(__name__)

In [4]:
# =====================================================================
# CONFIGURATION
# =====================================================================
@dataclass
class PipelineConfig:
    """Central configuration for the full pipeline."""
    # Crop
    bounds: Optional[Dict[str, float]] = None
    clip_polygon_path: Optional[str] = None

    # SOR
    sor_nb_neighbors: int = 20
    sor_std_ratio: float = 3.0
    sor_chunk_size: int = 5_000_000

    # Class filter
    valid_classes: List[int] = field(default_factory=lambda: [2, 3, 4, 5, 6])

    # HAG
    hag_k_neighbors: int = 4

    # --- Building reclassification (NEW) ---
    reclass_scales: List[int] = field(default_factory=lambda: [10, 20, 40])
    reclass_min_hag: float = 2.0            # metres — ignore anything below this
    reclass_max_hag: float = 50.0           # metres — ignore anything above this
    reclass_score_threshold: float = 0.55   # combined score to accept as building
    reclass_ransac_distance: float = 0.15   # RANSAC plane inlier distance (m)
    reclass_ransac_iterations: int = 100
    reclass_normal_std_threshold: float = 0.25  # max normal-Z std for building

    # Region growing
    rg_search_radius: float = 1.5     # metres
    rg_z_tolerance: float = 2.0       # metres
    rg_min_score: float = 0.40        # relaxed threshold for region-grown points
    rg_normal_agreement: float = 0.85 # cosine similarity of normals for growing

    # Cluster cleanup
    cluster_eps: float = 1.5
    cluster_min_size: int = 100
    use_hdbscan: bool = True


# =====================================================================
# MERGE (unchanged from previous upgrade)
# =====================================================================
def merge_las_files(paths: List[str], output_path: str) -> None:
    if len(paths) < 2:
        raise ValueError("Provide at least two files to merge.")

    log.info("Reading %d files for merge...", len(paths))
    datasets = [laspy.read(p) for p in paths]

    fmt_id = datasets[0].header.point_format.id
    for i, ds in enumerate(datasets[1:], start=1):
        if ds.header.point_format.id != fmt_id:
            raise ValueError(f"Point format mismatch: file 0={fmt_id}, file {i}={ds.header.point_format.id}")

    ref = datasets[0]
    new_header = laspy.LasHeader(point_format=ref.header.point_format, version=ref.header.version)
    new_header.offsets = ref.header.offsets
    new_header.scales = ref.header.scales
    new_header.vlrs = ref.header.vlrs

    merged_array = np.concatenate([ds.points.array for ds in datasets])
    merged_las = laspy.LasData(new_header)
    merged_las.points = laspy.ScaleAwarePointRecord(
        merged_array, ref.header.point_format, ref.header.scales, ref.header.offsets
    )
    merged_las.header.mins = np.array([merged_las.x.min(), merged_las.y.min(), merged_las.z.min()])
    merged_las.header.maxs = np.array([merged_las.x.max(), merged_las.y.max(), merged_las.z.max()])

    log.info("Writing %s merged points to %s", f"{len(merged_array):,}", output_path)
    merged_las.write(output_path)


# =====================================================================
# CROP (unchanged)
# =====================================================================
def crop_las_file(
    input_path: str, output_path: str,
    bounds: Optional[Dict[str, float]] = None,
    polygon_path: Optional[str] = None,
) -> None:
    las = laspy.read(input_path)
    log.info("Loaded %s points from %s", f"{len(las.points):,}", input_path)

    if polygon_path and HAS_SHAPELY:
        with fiona.open(polygon_path) as src:
            geom = shape(src[0]["geometry"])
        mask = contains(geom, las.x, las.y)
    elif bounds:
        mask = (
            (las.x >= bounds['min_x']) & (las.x <= bounds['max_x']) &
            (las.y >= bounds['min_y']) & (las.y <= bounds['max_y'])
        )
    else:
        raise ValueError("Provide either `bounds` dict or `polygon_path`.")

    cropped = las.points[mask]
    if len(cropped) == 0:
        log.error("No points within the specified region!")
        return

    new_header = laspy.LasHeader(point_format=las.header.point_format, version=las.header.version)
    new_header.offsets = las.header.offsets
    new_header.scales = las.header.scales
    new_header.vlrs = las.header.vlrs
    out = laspy.LasData(new_header)
    out.points = cropped
    log.info("Saving %s cropped points.", f"{len(cropped):,}")
    out.write(output_path)


# =====================================================================
# SOR + CLASS FILTER (unchanged)
# =====================================================================
def filter_lidar_data(
    input_path: str, output_path: str,
    nb_neighbors: int = 20, std_ratio: float = 2.0,
    valid_classes: Optional[List[int]] = None,
    chunk_size: int = 5_000_000,
) -> None:
    if valid_classes is None:
        valid_classes = [2, 3, 4, 5, 6]

    las = laspy.read(input_path)
    n_orig = len(las.points)
    points = np.vstack((las.x, las.y, las.z)).T

    log.info("Running chunked SOR on %s points (k=%d, σ=%.1f)...", f"{n_orig:,}", nb_neighbors, std_ratio)

    if len(points) <= chunk_size:
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(points)
        _, inlier_idx = pcd.remove_statistical_outlier(nb_neighbors=nb_neighbors, std_ratio=std_ratio)
        keep = np.zeros(len(points), dtype=bool)
        keep[inlier_idx] = True
    else:
        keep = np.ones(len(points), dtype=bool)
        x_sorted = np.argsort(points[:, 0])
        for start in range(0, len(points), chunk_size):
            end = min(start + chunk_size, len(points))
            cidx = x_sorted[start:end]
            pcd = o3d.geometry.PointCloud()
            pcd.points = o3d.utility.Vector3dVector(points[cidx])
            _, inlier_local = pcd.remove_statistical_outlier(nb_neighbors=nb_neighbors, std_ratio=std_ratio)
            outlier_local = np.setdiff1d(np.arange(len(cidx)), inlier_local)
            keep[cidx[outlier_local]] = False

    las = las[keep]
    log.info("SOR removed %s outliers.", f"{n_orig - len(las.points):,}")

    las = las[np.isin(las.classification, valid_classes)]
    log.info("After class filter: %s points.", f"{len(las.points):,}")
    las.write(output_path)


# =====================================================================
# HAG COMPUTATION (in-memory only — does NOT overwrite Z)
# =====================================================================
def _compute_hag(las, k: int = 4) -> np.ndarray:
    """
    Compute Height Above Ground for every point using IDW interpolation
    of the nearest ground (Class 2) points.

    Returns an (N,) array of HAG values in metres.  The LAS object's Z
    coordinates are **not modified** — original elevation is preserved.

    Parameters
    ----------
    las : laspy.LasData  — already-loaded point cloud (read once, reused).
    k   : int            — number of ground neighbours for IDW (default 4).
    """
    ground_mask = las.classification == 2
    if np.sum(ground_mask) == 0:
        log.warning("No Class 2 ground. Estimating from lowest 1%%...")
        sorted_idx = np.argsort(las.z)
        n_gnd = max(int(len(las.points) * 0.01), 10)
        ground_mask = np.zeros(len(las.points), dtype=bool)
        ground_mask[sorted_idx[:n_gnd]] = True

    gnd_xy = np.vstack((las.x[ground_mask], las.y[ground_mask])).T
    gnd_z = np.asarray(las.z[ground_mask], dtype=np.float64)
    log.info("Computing HAG from %s ground points (IDW k=%d)...", f"{len(gnd_z):,}", k)

    tree = cKDTree(gnd_xy)
    all_xy = np.vstack((las.x, las.y)).T
    dists, indices = tree.query(all_xy, k=k)

    if k == 1:
        z_ref = gnd_z[indices]
    else:
        dists = np.maximum(dists, 1e-10)
        w = 1.0 / dists
        w /= w.sum(axis=1, keepdims=True)
        z_ref = np.sum(w * gnd_z[indices], axis=1)

    hag = np.maximum(np.asarray(las.z, dtype=np.float64) - z_ref, 0.0)
    log.info("HAG range: %.2f – %.2f m", np.min(hag), np.max(hag))
    return hag


# =====================================================================
# GEOMETRIC FEATURE ENGINE (NEW)
# =====================================================================
def _compute_eigenfeatures_batch(
    coords: np.ndarray,
    neighbor_indices: np.ndarray,
) -> Dict[str, np.ndarray]:
    """
    Compute a full set of eigenvalue-derived geometric features for every point,
    fully vectorised (no Python loop).

    Returns dict with keys: planarity, sphericity, linearity, surface_variation,
    omnivariance, anisotropy, eigenentropy, normal_z, normal_z_std.

    Parameters
    ----------
    coords : (N, 3) candidate XYZ
    neighbor_indices : (N, k) indices into coords
    """
    N, k = neighbor_indices.shape
    neighbors = coords[neighbor_indices]                    # (N, k, 3)

    # --- Covariance ---
    means = neighbors.mean(axis=1, keepdims=True)           # (N, 1, 3)
    centered = neighbors - means                             # (N, k, 3)
    covs = np.einsum('nki,nkj->nij', centered, centered) / (k - 1)  # (N, 3, 3)

    # --- Eigendecomposition (ascending order) ---
    eigvals, eigvecs = np.linalg.eigh(covs)                 # (N,3), (N,3,3)

    # Clamp tiny negatives from numerical noise
    eigvals = np.maximum(eigvals, 0.0)
    e3, e2, e1 = eigvals[:, 0], eigvals[:, 1], eigvals[:, 2]
    esum = e1 + e2 + e3

    # Guard division
    safe_e1 = np.where(e1 > 0, e1, 1.0)
    safe_esum = np.where(esum > 0, esum, 1.0)

    # --- Eigenvalue features ---
    planarity = (e2 - e3) / safe_e1
    linearity = (e1 - e2) / safe_e1
    sphericity = e3 / safe_e1
    anisotropy = (e1 - e3) / safe_e1
    surface_variation = e3 / safe_esum
    omnivariance = np.cbrt(e1 * e2 * e3)

    # Eigenentropy — using normalised eigenvalues
    en = eigvals / safe_esum[:, None]
    en_safe = np.where(en > 0, en, 1.0)
    eigenentropy = -np.sum(en * np.log(en_safe), axis=1)

    # --- Normal vector features ---
    # The normal is the eigenvector corresponding to the smallest eigenvalue (column 0)
    normals = eigvecs[:, :, 0]  # (N, 3)
    # Make normals consistently upward-pointing
    flip = normals[:, 2] < 0
    normals[flip] *= -1
    normal_z = normals[:, 2]  # verticality: 1.0 = horizontal surface, 0.0 = vertical wall

    # Normal-Z standard deviation within neighbourhood: measures normal consistency
    # For each point, gather the normals of its k neighbours and compute std of their Z component
    neighbour_nz = normal_z[neighbor_indices]               # (N, k)
    normal_z_std = np.std(neighbour_nz, axis=1)             # (N,)

    return {
        'planarity': planarity,
        'linearity': linearity,
        'sphericity': sphericity,
        'anisotropy': anisotropy,
        'surface_variation': surface_variation,
        'omnivariance': omnivariance,
        'eigenentropy': eigenentropy,
        'normal_z': normal_z,
        'normal_z_std': normal_z_std,
    }


def _ransac_plane_inlier_ratio_batch(
    coords: np.ndarray,
    neighbor_indices: np.ndarray,
    distance_threshold: float = 0.15,
    n_iterations: int = 100,
    rng_seed: int = 42,
) -> np.ndarray:
    """
    For each point's neighbourhood, estimate the best-fit plane via RANSAC
    and return the fraction of neighbours that are inliers.

    Buildings → high inlier ratio (0.8–1.0)
    Vegetation → low inlier ratio (0.2–0.5)

    This is vectorised over the RANSAC iterations but loops over points
    because each has a different neighbourhood. Uses NumPy random sampling
    for speed.
    """
    rng = np.random.default_rng(rng_seed)
    N, k = neighbor_indices.shape
    inlier_ratios = np.zeros(N, dtype=np.float64)

    neighbors = coords[neighbor_indices]  # (N, k, 3)

    for i in range(N):
        pts = neighbors[i]  # (k, 3)
        best_inliers = 0

        for _ in range(n_iterations):
            # Sample 3 random points to define a plane
            idx3 = rng.choice(k, size=3, replace=False)
            p0, p1, p2 = pts[idx3[0]], pts[idx3[1]], pts[idx3[2]]

            # Plane normal via cross product
            v1 = p1 - p0
            v2 = p2 - p0
            normal = np.cross(v1, v2)
            norm_len = np.linalg.norm(normal)
            if norm_len < 1e-10:
                continue
            normal /= norm_len

            # Distance of all k points to the plane
            diffs = pts - p0
            distances = np.abs(diffs @ normal)

            n_inliers = np.sum(distances < distance_threshold)
            if n_inliers > best_inliers:
                best_inliers = n_inliers

        inlier_ratios[i] = best_inliers / k

    return inlier_ratios


def _compute_multi_scale_features(
    coords: np.ndarray,
    tree: cKDTree,
    scales: List[int],
    ransac_distance: float = 0.15,
    ransac_iterations: int = 100,
) -> Dict[str, np.ndarray]:
    """
    Compute geometric features at multiple neighbourhood scales and aggregate.

    For each scale k in `scales`, we compute the full eigenfeature set and
    RANSAC inlier ratio, then take the **maximum** planarity / inlier ratio
    across scales (to catch both large planar roofs and sharp edges) and the
    **minimum** normal_z_std (tightest normal consistency at any scale).

    This multi-scale approach is the key to handling:
    - Large flat roofs (captured at k=40)
    - Roof edges / ridgelines (captured at k=10)
    - Pitched roofs (good RANSAC fit even when planarity is moderate)
    """
    N = len(coords)

    # Accumulators — we'll stack across scales and then reduce
    all_planarity = []
    all_linearity = []
    all_sphericity = []
    all_surface_var = []
    all_normal_z = []
    all_normal_z_std = []
    all_ransac = []

    for k_scale in scales:
        log.info("  Computing features at scale k=%d...", k_scale)
        _, ind = tree.query(coords, k=k_scale)

        feats = _compute_eigenfeatures_batch(coords, ind)
        ransac_ratio = _ransac_plane_inlier_ratio_batch(
            coords, ind,
            distance_threshold=ransac_distance,
            n_iterations=ransac_iterations,
        )

        all_planarity.append(feats['planarity'])
        all_linearity.append(feats['linearity'])
        all_sphericity.append(feats['sphericity'])
        all_surface_var.append(feats['surface_variation'])
        all_normal_z.append(feats['normal_z'])
        all_normal_z_std.append(feats['normal_z_std'])
        all_ransac.append(ransac_ratio)

    # Stack: (n_scales, N)
    planarity_stack = np.stack(all_planarity)
    linearity_stack = np.stack(all_linearity)
    sphericity_stack = np.stack(all_sphericity)
    sv_stack = np.stack(all_surface_var)
    nz_stack = np.stack(all_normal_z)
    nz_std_stack = np.stack(all_normal_z_std)
    ransac_stack = np.stack(all_ransac)

    return {
        # Take best (most building-like) value across scales
        'planarity_max': np.max(planarity_stack, axis=0),
        'linearity_min': np.min(linearity_stack, axis=0),
        'sphericity_min': np.min(sphericity_stack, axis=0),
        'surface_var_min': np.min(sv_stack, axis=0),
        'normal_z_max': np.max(nz_stack, axis=0),
        'normal_z_std_min': np.min(nz_std_stack, axis=0),
        'ransac_inlier_max': np.max(ransac_stack, axis=0),

        # Also keep per-scale for diagnostics
        '_planarity_per_scale': planarity_stack,
        '_ransac_per_scale': ransac_stack,
    }


# =====================================================================
# BUILDING SCORING FUNCTION (NEW)
# =====================================================================
def _compute_building_score(features: Dict[str, np.ndarray]) -> np.ndarray:
    """
    Combine multiple geometric features into a single building confidence
    score in [0, 1] using a weighted sum.

    Each feature is mapped to [0, 1] with a sigmoid-like soft threshold,
    then weighted by its discriminative importance. Weights are derived
    from empirical analysis of Nordic ALS data but are reasonable defaults
    for most urban/suburban scenes.

    Feature contributions:
        planarity_max     (weight 0.25) — core geometric cue
        ransac_inlier_max (weight 0.30) — most robust single feature
        normal_z_std_min  (weight 0.20) — penalises vegetation strongly
        sphericity_min    (weight 0.10) — penalises scattered volumes
        surface_var_min   (weight 0.15) — penalises rough surfaces

    Returns (N,) float array of building confidence scores.
    """
    def _sigmoid(x, center, steepness):
        """Soft threshold: maps x to [0, 1] centered at `center`."""
        return 1.0 / (1.0 + np.exp(-steepness * (x - center)))

    # Planarity: high is good. Center at 0.4, steep transition.
    s_plan = _sigmoid(features['planarity_max'], center=0.4, steepness=12.0)

    # RANSAC inlier ratio: high is good. Center at 0.65.
    s_ransac = _sigmoid(features['ransac_inlier_max'], center=0.65, steepness=15.0)

    # Normal-Z std: LOW is good (consistent normals). Invert.
    s_nstd = 1.0 - _sigmoid(features['normal_z_std_min'], center=0.20, steepness=20.0)

    # Sphericity: LOW is good (not a ball-shaped canopy). Invert.
    s_sph = 1.0 - _sigmoid(features['sphericity_min'], center=0.3, steepness=10.0)

    # Surface variation: LOW is good. Invert.
    s_sv = 1.0 - _sigmoid(features['surface_var_min'], center=0.15, steepness=15.0)

    score = (
        0.25 * s_plan +
        0.30 * s_ransac +
        0.20 * s_nstd +
        0.10 * s_sph +
        0.15 * s_sv
    )
    return score


# =====================================================================
# REGION GROWING REFINEMENT (NEW — replaces recover_roof_edges)
# =====================================================================
def _region_grow_buildings(
    coords: np.ndarray,
    scores: np.ndarray,
    seed_mask: np.ndarray,
    candidate_mask: np.ndarray,
    normals_z: np.ndarray,
    search_radius: float = 1.5,
    z_tolerance: float = 2.0,
    min_score: float = 0.40,
    normal_agreement: float = 0.85,
) -> np.ndarray:
    """
    Region-growing from confirmed building seeds into adjacent candidates.

    This replaces the separate "recover_roof_edges" + "filter_building_outliers"
    two-step with a single principled pass that:
      1. Starts from high-confidence building seeds.
      2. Examines all non-seed candidates within `search_radius`.
      3. Accepts a candidate if:
         a. Its building score >= `min_score` (relaxed vs. seed threshold), AND
         b. Its Z is within `z_tolerance` of the seed, AND
         c. Its surface normal is consistent with the seed (cosine > `normal_agreement`
            measured via the Z-component, since we're comparing flatness).
      4. Newly accepted points become seeds for the next iteration.
      5. Repeats until no more points are added.

    This naturally captures roof edges (which have moderate scores but are
    spatially and normally consistent with the roof) while rejecting isolated
    vegetation (which fails the spatial + normal consistency checks).

    Returns updated boolean mask of all building points.
    """
    building_mask = seed_mask.copy()
    remaining = candidate_mask & ~seed_mask

    tree = cKDTree(coords)

    iteration = 0
    while True:
        iteration += 1
        current_building_idx = np.where(building_mask)[0]
        current_remaining_idx = np.where(remaining)[0]

        if len(current_remaining_idx) == 0:
            break

        # For each remaining candidate, find nearest building point
        remaining_coords = coords[current_remaining_idx]
        building_coords = coords[current_building_idx]

        if len(building_coords) == 0:
            break

        btree = cKDTree(building_coords)
        dists, nearest_bldg_local = btree.query(remaining_coords, k=1)

        # Apply acceptance criteria — fully vectorised
        nearest_bldg_global = current_building_idx[nearest_bldg_local]

        cond_dist = dists <= search_radius
        cond_z = np.abs(coords[current_remaining_idx, 2] - coords[nearest_bldg_global, 2]) <= z_tolerance
        cond_score = scores[current_remaining_idx] >= min_score
        cond_normal = np.abs(normals_z[current_remaining_idx] - normals_z[nearest_bldg_global]) <= (1.0 - normal_agreement)

        accept = cond_dist & cond_z & cond_score & cond_normal
        accepted_global = current_remaining_idx[accept]

        if len(accepted_global) == 0:
            break

        building_mask[accepted_global] = True
        remaining[accepted_global] = False
        log.info("  Region grow iter %d: added %d points", iteration, len(accepted_global))

    return building_mask


# =====================================================================
# MAIN RECLASSIFICATION FUNCTION (NEW — replaces 3 old functions)
# =====================================================================
def reclassify_buildings(
    input_path: str,
    output_path: str,
    cfg: PipelineConfig,
) -> None:
    """
    Multi-feature, multi-scale building reclassification with region growing.

    Replaces the old three-function chain:
        reclassify_buildings_from_veg → filter_building_outliers → recover_roof_edges

    HAG is computed in-memory and used ONLY for candidate filtering.
    The output file retains the original elevation Z values.

    Pipeline:
        1. Compute HAG in-memory (original Z untouched)
        2. HAG filter — exclude ground and extreme outliers
        3. Multi-scale eigenfeatures + RANSAC on candidates (using original elevation)
        4. Weighted score computation
        5. High-confidence seeds (score > threshold)
        6. Region-growing to capture edges
        7. Cluster-based cleanup of remaining noise
        8. Write output with original elevation preserved
    """
    t0 = time.perf_counter()
    las = laspy.read(input_path)
    log.info("Loaded %s points for building reclassification.", f"{len(las.points):,}")

    # Original elevation coordinates — used for ALL geometry and preserved in output
    coords = np.vstack((las.x, las.y, las.z)).T
    classification = np.asarray(las.classification).copy()

    # ------------------------------------------------------------------
    # STEP 1: Compute HAG in-memory (Z is NOT overwritten)
    # ------------------------------------------------------------------
    hag = _compute_hag(las, k=cfg.hag_k_neighbors)

    # ------------------------------------------------------------------
    # STEP 2: HAG-based candidate selection
    # ------------------------------------------------------------------
    # Use the in-memory HAG array to filter candidates by height,
    # but all subsequent geometry uses the original elevation coords.
    candidate_mask = (
        np.isin(classification, [1, 5]) &   # Unclassified or High Vegetation
        (hag >= cfg.reclass_min_hag) &
        (hag <= cfg.reclass_max_hag)
    )

    candidate_indices = np.where(candidate_mask)[0]
    log.info("Step 2 — HAG filter: %s candidates (%.1f–%.1f m above ground)",
             f"{len(candidate_indices):,}", cfg.reclass_min_hag, cfg.reclass_max_hag)

    if len(candidate_indices) == 0:
        log.warning("No candidates after HAG filter. Check min_hag / max_hag settings.")
        las.write(output_path)
        return

    # Use original-elevation coords for all geometric analysis
    cand_coords = coords[candidate_indices]

    # ------------------------------------------------------------------
    # STEP 3: Multi-scale geometric features
    # ------------------------------------------------------------------
    log.info("Step 3 — Computing multi-scale features at k=%s...", cfg.reclass_scales)
    tree = cKDTree(cand_coords)
    ms_features = _compute_multi_scale_features(
        cand_coords, tree,
        scales=cfg.reclass_scales,
        ransac_distance=cfg.reclass_ransac_distance,
        ransac_iterations=cfg.reclass_ransac_iterations,
    )

    # ------------------------------------------------------------------
    # STEP 4: Building score
    # ------------------------------------------------------------------
    log.info("Step 4 — Computing building confidence scores...")
    scores = _compute_building_score(ms_features)

    log.info("  Score distribution: min=%.3f  median=%.3f  mean=%.3f  max=%.3f",
             np.min(scores), np.median(scores), np.mean(scores), np.max(scores))

    # ------------------------------------------------------------------
    # STEP 5: High-confidence seeds
    # ------------------------------------------------------------------
    seed_local_mask = scores >= cfg.reclass_score_threshold
    n_seeds = int(np.sum(seed_local_mask))
    log.info("Step 4 — Seeds (score >= %.2f): %s points",
             cfg.reclass_score_threshold, f"{n_seeds:,}")

    if n_seeds == 0:
        log.warning("No seeds found. Consider lowering reclass_score_threshold.")
        las.write(output_path)
        return

    # ------------------------------------------------------------------
    # STEP 6: Region growing
    # ------------------------------------------------------------------
    log.info("Step 6 — Region growing (radius=%.1f m, z_tol=%.1f m, min_score=%.2f)...",
             cfg.rg_search_radius, cfg.rg_z_tolerance, cfg.rg_min_score)

    # We need normal_z for the agreement check
    # Use the largest scale for the most stable normals
    max_k = max(cfg.reclass_scales)
    _, ind_norms = tree.query(cand_coords, k=max_k)
    norm_feats = _compute_eigenfeatures_batch(cand_coords, ind_norms)
    normals_z = norm_feats['normal_z']

    all_candidate_local = np.ones(len(cand_coords), dtype=bool)
    building_local_mask = _region_grow_buildings(
        coords=cand_coords,
        scores=scores,
        seed_mask=seed_local_mask,
        candidate_mask=all_candidate_local,
        normals_z=normals_z,
        search_radius=cfg.rg_search_radius,
        z_tolerance=cfg.rg_z_tolerance,
        min_score=cfg.rg_min_score,
        normal_agreement=cfg.rg_normal_agreement,
    )

    n_after_rg = int(np.sum(building_local_mask))
    log.info("  After region growing: %s building points (grew %s from seeds)",
             f"{n_after_rg:,}", f"{n_after_rg - n_seeds:,}")

    # ------------------------------------------------------------------
    # STEP 7: Cluster cleanup — remove tiny isolated clusters
    # ------------------------------------------------------------------
    log.info("Step 7 — Cluster-based cleanup (min_size=%d)...", cfg.cluster_min_size)
    bldg_local_idx = np.where(building_local_mask)[0]

    if len(bldg_local_idx) > 0:
        bldg_coords = cand_coords[bldg_local_idx]

        if cfg.use_hdbscan and HAS_HDBSCAN:
            clusterer = HDBSCAN(min_cluster_size=cfg.cluster_min_size, min_samples=10)
            labels = clusterer.fit_predict(bldg_coords)
        else:
            labels = DBSCAN(eps=cfg.cluster_eps, min_samples=10).fit_predict(bldg_coords)

        unique_labels, counts = np.unique(labels, return_counts=True)
        size_map = dict(zip(unique_labels, counts))
        point_sizes = np.array([size_map[l] for l in labels])

        valid = (labels != -1) & (point_sizes >= cfg.cluster_min_size)
        building_local_mask[bldg_local_idx[~valid]] = False

        n_reverted = int(np.sum(~valid))
        n_final = int(np.sum(building_local_mask))
        n_valid_clusters = int(np.sum((unique_labels != -1) & (counts >= cfg.cluster_min_size)))
        log.info("  Cleanup: reverted %d spurious points, kept %d in %d clusters.",
                 n_reverted, n_final, n_valid_clusters)

    # ------------------------------------------------------------------
    # APPLY to LAS classification
    # ------------------------------------------------------------------
    building_global_idx = candidate_indices[building_local_mask]
    classification[building_global_idx] = 6
    las.classification = classification

    las.write(output_path)
    log.info("Building reclassification complete in %.1f s. Saved to %s",
             time.perf_counter() - t0, output_path)


# =====================================================================
# VISUALIZATION (expanded)
# =====================================================================
def visualize_classification(las_file_path: str) -> None:
    las = laspy.read(las_file_path)
    points = np.vstack((las.x, las.y, las.z)).T
    classification = np.asarray(las.classification)

    color_map = {
        2: [0.6, 0.4, 0.2],   # Ground
        3: [0.0, 0.9, 0.0],   # Low Veg
        4: [0.0, 0.7, 0.0],   # Med Veg
        5: [0.0, 0.5, 0.0],   # High Veg
        6: [1.0, 0.0, 0.0],   # Building
    }
    colors = np.tile([0.5, 0.5, 0.5], (len(points), 1))
    for cls, rgb in color_map.items():
        colors[classification == cls] = rgb

    for cls in sorted(color_map):
        c = int(np.sum(classification == cls))
        if c > 0:
            log.info("Class %d: %s points", cls, f"{c:,}")

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    pcd.colors = o3d.utility.Vector3dVector(colors)

    vis = o3d.visualization.Visualizer()
    vis.create_window(window_name="LiDAR Classification Viewer")
    vis.add_geometry(pcd)
    opt = vis.get_render_option()
    opt.background_color = np.asarray([0, 0, 0])
    vis.run()
    vis.destroy_window()


# =====================================================================
# PIPELINE
# =====================================================================
def run_pipeline(cfg: PipelineConfig) -> None:
    """
    Upgraded pipeline order:
        1. Crop
        2. SOR + class filter
        3. Building reclassification (computes HAG internally, outputs original elevation)
        4. Visualise
    """
    input_laz = "data/study_area.laz"
    cropped = "data/cropped.laz"
    filtered = "output/filtered.laz"
    reclassified = "output/reclassified_final.laz"

    os.makedirs("output", exist_ok=True)
    t0 = time.perf_counter()

    crop_las_file(input_laz, cropped, bounds=cfg.bounds)
    filter_lidar_data(cropped, filtered,
                      nb_neighbors=cfg.sor_nb_neighbors,
                      std_ratio=cfg.sor_std_ratio,
                      valid_classes=cfg.valid_classes,
                      chunk_size=cfg.sor_chunk_size)

    # Reclassification computes HAG in-memory for candidate filtering,
    # but the output file retains the original elevation Z values.
    reclassify_buildings(filtered, reclassified, cfg=cfg)

    log.info("Full pipeline finished in %.1f s.", time.perf_counter() - t0)
    visualize_classification(reclassified)

In [5]:
# =====================================================================
# MAIN
# =====================================================================

target_x, target_y = 532885, 6983510

config = PipelineConfig(
    bounds={
        'min_x': target_x - 150,
        'max_x': target_x + 350,
        'min_y': target_y - 350,
        'max_y': target_y + 150,
    },
    # SOR
    sor_nb_neighbors=20,
    sor_std_ratio=3.0,
    # HAG
    hag_k_neighbors=4,
    # Building reclassification
    reclass_scales=[10, 20, 40],
    reclass_min_hag=2.0,
    reclass_max_hag=50.0,
    reclass_score_threshold=0.55,
    reclass_ransac_distance=0.15,
    reclass_ransac_iterations=100,
    reclass_normal_std_threshold=0.25,
    # Region growing
    rg_search_radius=1.5,
    rg_z_tolerance=2.0,
    rg_min_score=0.40,
    rg_normal_agreement=0.85,
    # Cluster cleanup
    cluster_min_size=100,
    use_hdbscan=True,
)

run_pipeline(config)

2026-02-21 00:51:47,363 [INFO] Loaded 15,413,412 points from data/study_area.laz
2026-02-21 00:51:47,476 [INFO] Saving 2,328,282 cropped points.
2026-02-21 00:51:47,645 [INFO] Running chunked SOR on 2,328,282 points (k=20, σ=3.0)...
2026-02-21 00:51:49,535 [INFO] SOR removed 30,851 outliers.
2026-02-21 00:51:49,609 [INFO] After class filter: 2,297,097 points.
2026-02-21 00:51:49,765 [INFO] Loaded 2,297,097 points for building reclassification.
2026-02-21 00:51:49,801 [INFO] Computing HAG from 793,227 ground points (IDW k=4)...
2026-02-21 00:51:51,370 [INFO] HAG range: 0.00 – 33.15 m
2026-02-21 00:51:51,381 [INFO] Step 2 — HAG filter: 1,251,288 candidates (2.0–50.0 m above ground)
2026-02-21 00:51:51,388 [INFO] Step 3 — Computing multi-scale features at k=[10, 20, 40]...
2026-02-21 00:51:51,583 [INFO]   Computing features at scale k=10...


KeyboardInterrupt: 